In [1]:
import numpy as np
import sys
import os
import pandas as pd
import plotly.graph_objects as go

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from levenberg_marquadt import levenberg_marquadt
from normalize import StandardScaler
from loss_fn_tarefa4 import make_mse_loss_for_neuron

pd.set_option('display.float_format', '{:.5f}'.format)

# Ajuste de curva por otimização

## Carregar os dados

In [2]:
# Carregamento dos dados
df = pd.read_excel('../data/Trabalho4dados.xlsx')

df.head()

,x1,x2,y
0,-5.00000,-3.00000,-0.91722
1,-4.89899,-2.93939,-0.93968
2,-4.79798,-2.87879,-0.91822
3,-4.69697,-2.81818,-0.90445
4,-4.59596,-2.75758,-0.93496


## EDA dados

In [3]:
df.describe()

,x1,x2,y
count,100.00000,100.00000,100.00000
mean,0.00000,-0.00000,0.27400
std,2.93045,1.75827,0.93269
min,-5.00000,-3.00000,-0.99989
25%,-2.50000,-1.50000,-0.91039
50%,0.00000,0.00000,1.00238
75%,2.50000,1.50000,1.05173
max,5.00000,3.00000,1.09797


In [4]:
df.isna().sum()

x1    0
x2    0
y     0
dtype: int64

In [5]:
df_pivot = df.pivot(index='x2', columns='x1', values='y')

fig = go.Figure()

# Adicionar os Pontos de Dados Originais
fig.add_trace(go.Scatter3d(
    x=df['x1'],
    y=df['x2'],
    z=df['y'],
    mode='markers',
    marker=dict(size=3, color='red', symbol='circle'),
    name='Pontos de Dados Originais'
))

# Melhorar o Layout
fig.update_layout(
    title=dict(text='y = f(x1, x2)', x=0.5),
    scene=dict(
        xaxis_title='Eixo X1',
        yaxis_title='Eixo X2',
        zaxis_title='Eixo Y (Valor)'
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

fig.show()

## Calcular as funções de perda

### Funções utilizadas para os cálculos

In [6]:
def tanh_derivative(x):
    """Derivada de tanh(x) = 1 - tanh(x)^2"""
    t = np.tanh(x)
    return 1 - t**2


def make_neuron(w, activation_fn=np.tanh):
    def neuron(x1, x2):
        # Converte para arrays para suportar tanto escalar quanto vetor
        x1 = np.atleast_1d(x1)
        x2 = np.atleast_1d(x2)

        # Matriz de entradas com bias = 1
        X = np.stack([x1, x2, np.ones_like(x1)], axis=1)

        # Potencial de ativação
        activation_potentials = X @ w  # produto matricial

        # Saída após ativação
        y = activation_fn(activation_potentials)

        # Se a entrada era escalar, devolve escalar
        return y if len(y) > 1 else y.item()
    return neuron


def make_residuals_fn(X1: np.ndarray, X2: np.ndarray, Y: np.ndarray, activation_fn=np.tanh):
    def residuals_fn(w: np.ndarray) -> np.ndarray:
        neuron = make_neuron(w, activation_fn)
        predictions = neuron(X1, X2)
        return Y - predictions
    return residuals_fn


def make_jacobian_fn(X1, X2, activation_fn=np.tanh, activation_deriv=tanh_derivative):
    """
    Jacobiana genérica: derivada dos resíduos em relação aos pesos.
    Funciona para qualquer função de ativação.
    """
    def jacobian_fn(w):
        X1_ = np.atleast_1d(X1)
        X2_ = np.atleast_1d(X2)
        X = np.stack([X1_, X2_, np.ones_like(X1_)], axis=1)
        z = X @ w
        phi_prime = activation_deriv(z)
        J = -phi_prime[:, np.newaxis] * X  # cada linha: -φ'(z_i) * [x1, x2, 1]
        return J
    return jacobian_fn

### Pré-processamento dos dados

In [7]:

features = df[['x1', 'x2']]
y = df['y']

# Criar os objetos para Padronização
scaler = StandardScaler()
scaler_y = StandardScaler()

# Cria as cópias dos dados para padronizar
scaled_features = features.copy()
scaled_y = y.copy()

# Ajusta os padronizadores aos dados
scaler.fit(scaled_features)
scaler_y.fit(scaled_y.to_frame())

# Padroniza os dados
scaled_features = scaler.normalize(scaled_features)
scaled_y = scaler_y.normalize(scaled_y.to_frame()).squeeze()

initial_weights = np.random.randn(3)  # 2 entradas + 1 bias
n_iterations = 10000
tolerance = 1e-6
alpha = 1e-3

### Rodar os experimentos

In [8]:
dict_results = {}

x1_scaled = scaled_features['x1'].values
x2_scaled = scaled_features['x2'].values
y_scaled_data = scaled_y.values

# Função de custo e gradiente
loss_function, grad_loss_function = make_mse_loss_for_neuron(x1_scaled, x2_scaled, y_scaled_data, activation_fn=np.tanh)

# Função de residuos e jacobiana
residuals_fn = make_residuals_fn(x1_scaled, x2_scaled, y_scaled_data, activation_fn=np.tanh)
jacobian_fn = make_jacobian_fn(x1_scaled, x2_scaled, activation_fn=np.tanh, activation_deriv=tanh_derivative)

# Treinar o modelo com Levenberg-Marquadt
weights, losses, n_iters = levenberg_marquadt(
    initial_weights, residuals_fn, loss_function, jacobian_fn,
    alpha=alpha, alpha_variability=10, max_iter=n_iterations,
    tolerance=tolerance, stopping_criteria=[1, 3]
)

# Desnormalizar os pesos finais
final_weights_raw = weights[-1].copy()
final_weights_denorm = scaler.desnormalize_weights(weights[-1])

# Usar o neurônio com pesos desnormalizados
x1_data = features['x1'].values
x2_data = features['x2'].values
neuron = make_neuron(final_weights_denorm, activation_fn=np.tanh)
y_hat = neuron(x1_data, x2_data)

# Cálculo das métricas finais
mse_final = np.mean((y - y_hat) ** 2)
rmse_final = np.sqrt(mse_final)
mae_final = np.mean(np.abs(y - y_hat))

dict_results = {
    'Feature_Set': "Standardized",
    'Loss_Function': "MSE",
    'Initial_Weights': str([f"{w:.5f}" for w in initial_weights.tolist()]),
    'Weights_Raw': str([f"{w:.5f}" for w in final_weights_raw.tolist()]),
    'Final_Weights': str([f"{w:.5f}" for w in final_weights_denorm.tolist()]),
    'Final_Loss': losses[-1],
    'MSE_Final': mse_final,
    'RMSE_Final': rmse_final,
    'MAE_Final': mae_final,
    'Iterations': n_iters
}

# Criar DataFrame com um único registro
df_result = pd.DataFrame([dict_results])  # Note os colchetes extras aqui
df_result

,Feature_Set,Loss_Function,Initial_Weights,Weights_Raw,Final_Weights,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations
0,Standardized,MSE,"['-1.42848', '0.87861', '1.32728']","['2.03713', '4.34423', '2.10975']","['0.69866', '2.48318', '2.10975']",0.05313,0.01846,0.13585,0.08375,10


## Resultados

In [9]:
# Criar figura 3D
fig = go.Figure()

# Adicionar pontos originais
fig.add_trace(go.Scatter3d(
    x=x1_data,
    y=x2_data,
    z=y,
    mode='markers',
    marker=dict(
        size=5,
        color='red',
        symbol='circle'
    ),
    name='Dados Originais'
))

# Adicionar pontos previstos
fig.add_trace(go.Scatter3d(
    x=x1_data,
    y=x2_data,
    z=y_hat,
    mode='markers',
    marker=dict(
        size=5,
        color='blue',
        symbol='diamond'
    ),
    name='Valores Previstos'
))

# Adicionar linhas verticais conectando pontos originais e previsões
for i in range(len(x1_data)):
    fig.add_trace(go.Scatter3d(
        x=[x1_data[i], x1_data[i]],
        y=[x2_data[i], x2_data[i]],
        z=[y[i], y_hat[i]],
        mode='lines',
        line=dict(color='green', width=2),
        showlegend=False
    ))

# Configurar layout
fig.update_layout(
    title='Comparação entre Valores Reais e Previstos',
    scene=dict(
        xaxis_title='x1',
        yaxis_title='x2',
        zaxis_title='y',
        aspectmode='data'
    ),
    width=900,
    height=700,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255,255,255,0.8)"
    )
)

pesos_info = (f"Pesos finais: {[f'{w:.5f}' for w in final_weights_denorm]}<br>"
              f"MSE: {mse_final:.5f}, RMSE: {rmse_final:.5f}<br>"
              f"Iterações: {n_iters}")

fig.add_annotation(
    x=0.5, y=0.01,
    xref="paper", yref="paper",
    text=pesos_info,
    showarrow=False,
    font=dict(size=12),
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="black",
    borderwidth=1,
    borderpad=4
)

fig.show()